In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

2022-10-23 21:58:53.788865: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 21:58:54.110317: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-23 21:58:54.117651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 21:58:54.117673: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# Take the sign names in our collections data

In [3]:
signs = os.listdir("./Model_Data/") # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

# Create the our x and y data


In [4]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [ ]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [ ]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [55]:
model.fit(x_train, y_train, epochs=93) # run this line when data collection is done (this will train the model)
# we can start the number of epochs to be equal to 175 to avoid over training and add afterwards if needed

Epoch 1/95
12/12 [==============================] - 3s 50ms/step - loss: 2.5917 - categorical_accuracy: 0.0703
Epoch 2/95
12/12 [==============================] - 1s 46ms/step - loss: 2.6422 - categorical_accuracy: 0.0649
Epoch 3/95
12/12 [==============================] - 1s 47ms/step - loss: 2.5565 - categorical_accuracy: 0.1351
Epoch 4/95
12/12 [==============================] - 1s 46ms/step - loss: 2.5056 - categorical_accuracy: 0.1162
Epoch 5/95
12/12 [==============================] - 1s 46ms/step - loss: 2.3433 - categorical_accuracy: 0.1541
Epoch 6/95
12/12 [==============================] - 1s 47ms/step - loss: 2.1971 - categorical_accuracy: 0.1865
Epoch 7/95
12/12 [==============================] - 1s 47ms/step - loss: 2.0218 - categorical_accuracy: 0.2189
Epoch 8/95
12/12 [==============================] - 1s 47ms/step - loss: 1.9687 - categorical_accuracy: 0.2811
Epoch 9/95
12/12 [==============================] - 1s 46ms/step - loss: 1.7905 - categorical_accuracy: 0.3405
E

In [56]:
model.summary() # summary of our model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 30, 64)            49408     
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_9 (Dense)             (None, 32)                61472     
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dense_11 (Dense)            (None, 13)               

In [57]:
res = model.predict(x_test)

1/1 [==============================] - 0s 353ms/step


In [76]:
signs[np.argmax(res[10])]

'how are you'

In [75]:
signs[np.argmax(y_test[10])]

'how are you'

# save weights


In [77]:
model.save("VertoMotus_MLmodel.h5") # save model